# Write - buy options for Covered call Strategy

<b>Purpose</b> - For a given stock position and option expiration date ,  this algorithm will find the best set of combinations of contract sizes, strike prices and bid prices which yield maximum returns at different maturity prices.

At maturity price st, expected return function is formulated as , 

<b>Minimize - (𝑠𝑡*N−𝑠0*N+∑(𝑝𝑗(𝐶𝑗∗𝑟𝑓 −max((𝑠𝑡−𝑘𝑗),0))/𝑠0*N</b>

<b>Subject to ∑𝑝𝑗≤𝑁 𝑎𝑛𝑑 𝑁≥1 </b>


Where   st - Maturity price at the time of option expiration
        s0 - Current Market price
        pj - Contract sizes
        Cj - bid prices
        kj - Strike prices
        rf - risk free rate

<b>Output</b> - Optimum set of pj, kj, Cj (Contract size, Strike price, Bid price) for different maturity prices st1, st2,..stN

<b>Input parameters are --</b>

1. Stock Symbol
2. Stock Position (Input stock position should be greater than 500 due to the limitation that decision parameter is 'n' 
     -- yet to work on finding best 'n' automatically)
3. Option Expiration Date (Optional) - If not provided, latest option expiration date is considered

<b>Decision parameters</b> - 'n' is the grouping of given stock position into Contract sizes, strike prices, bid prices. In the algorithm,  n can be in the range (1, number of units of stock position). There is no best 'n'.

For now i have hard coded it as 5, however best 'n' is yet to be identified. 
n= 5 means, if input stock position is 1000 which is 10 units (1 unit is 100 shares). output of the algorithm distribute 10 units into 5 sets with varying contract sizes, strike price and bid prices

The output data for n=5 for different maturity prices are-

        Contract Sizes    strike prices           bid prices                         Expected Return         Maturity price
        [1, 1, 1, 1, 6]   [70. 75. 80. 85. 90.]   [52.95 48.1  43.8  38.8  33.8 ]    -3.14770487541152       124.61
        [1, 1, 1, 1, 6]   [70. 75. 80. 85. 90.]   [52.95 48.1  43.8  38.8  33.8 ]    -3.2792352447846        126.43
        ... so on

Total number of contract sizes is equal to number of units from given stock position.
Expected return is shown as negative here. Becasue i have formulated optimization problem 

Maximize Expexted Returns as Minimize Negative Expected returns


<u>Alternative option 'n' :</u>

'n' can be made as input parameter to the algorithm

<b>Steps:</b>

<b>Data Extraction -</b>

1. For the given stock symbol, extract current market price s0
2. For the given stock symbol and option expiration date, extract only OTM (Out of the money) call option data whose bid value exists. If Expiration date is not provided as input, take the latest expiration date from option expiration dates and extract call options.
3. Get the strike prices and bid prices from step 2.
4. Select N number of Maturity prices 'st' ranging from Current market price-100 to Maximum strike price from Call option data from step2 (N can be selected as many as possible)
5. Set Risk free rate (at the moment assigned to 1) 

<b>Algorithm -</b>
1. Repeat from 1 to N from step 1 to d

   a) Initialized random contract sizes as starting point to the algorithm. 
   b) Set up Contraint as Total number of Contracts should not exceed number of units of stock position
   c) Set up bound for contract size min = 1 unit max = Number of units of stock position
   d) Run Objective function with the arguments (s0, st, k, C and risk free rate) get optimum contract sizes with maximum returns
   
2. Output optimum values(Contract sizes, Strike prices, bid prices, returns, Maturity price) from 1 to N maturity prices

<b>Result:</b> Contract sizes, Strike prices, bid prices, returns, Maturity price. Example is 


        Contract Sizes    strike prices           bid prices                         Expected Return         Maturity price
        [1, 1, 1, 1, 6]   [70. 75. 80. 85. 90.]   [52.95 48.1  43.8  38.8  33.8 ]    -3.14770487541152       124.61
        [1, 1, 1, 1, 6]   [70. 75. 80. 85. 90.]   [52.95 48.1  43.8  38.8  33.8 ]    -3.2792352447846        126.43
        ... so on

<b>Limitations</b> - 

1. Input Stock position should be greater or equal to 500 
2. 'n' size is not calculated automatically. 
3. Dividends are not taken into account

In [4]:
# Import necessary libraries
import sys, getopt
import argparse
import time
import yfinance as yf # Import yahoo finance module
import pandas as pd
from datetime import date
from scipy.optimize import minimize, LinearConstraint
import numpy as np
import random


In [65]:
'''
# This class has 
     Input      :- 
                     stocksymbol             (String)
                     currentstockposition  (integer)
                     option_expiration_date (date)
                        
     Output :-
                     contract size
                     strike price
                     bid price
                     expiration date    (next earliest date from the current date)

'''

class MaximizeMarginUtilization:
    def __init__(self, arguments):
        
        print(arguments)
        self.symbol = arguments['stksymbol']
        self.N = arguments['stkposition']
        self.exp_date = arguments['expdate']     
       
    def run(self):
                
                
        #OPTIMISATION USING SCIPY
        def get_current_price(self):  
            # This procedure is used to pick Current stock price, which is the last closing price of the day
            stk = yf.Ticker(self.symbol) # Passing stock stymbol
            todays_data = stk.history(period='1d')
            return todays_data['Close'][0]
        
        def Objective_func(x, st, s0, diff, C, N):
    
            # is contract size            
            
            # risk free is considered as 1 for calculation as there is no clarity on how this rate is decided.
            rf = 1 
            
            # Objective function (Si − S0 + pj(Cj *rf − SiT − kj))/s0  
            # Maximization = -1 * Minimization
            #f = -1*(((st*N)-(s0*N)+np.sum(p*((C*rf) - diff)))*100/(s0*N)) 
            f = -1*((st-s0+np.sum(x*((C*rf) - diff)))*100/s0) 
            
            # function value f is the percentage of profit obtained for N units out of current value of the stock in units
            # i.e., if N is 5 units and current stock price is 100$, current value is N*100$
            # if optimum f value is -4 that means profit is 4% of 100$*N. 
            # if optimum f value is positive, that means loss %  of current value of the assets
            # if optimum f value is negative, that means profit % of current value of the assets
            #prob = [int(round(p,0)) for p in p]
            print(N, x, diff, C, s0, st, -((f/100)*(s0*N) )) # print all iterations
            
            
                                   
            return f    # return negative of best minimum return , which means maximum return
        
        def constrained_sum_sample_pos(n, total):
            """Return a randomly chosen list of n positive integers summing to total.
            Each such list is equally likely to occur."""

            dividers = sorted(random.sample(range(1, total), n - 1))
            return [a - b for a, b in zip(dividers + [total], [0] + dividers)]
        
        # Number of groups of contract sizes. 
        #n can be of any number from 1 to number of units of assets which is input for this algorithm     
        n = 5    
        
        # Current market price is today's last closing price            
        s0 = get_current_price(self) 
        
        # Getting stock options from Yahoo Finance
        stk = yf.Ticker(self.symbol) # Passing stock stymbol to get stock options
                
        if self.exp_date > '' and self.exp_date in stk.options: # Check if option expiration date is given as input
            print('Input Call Option Expiration Date', self.exp_date)
            opt = stk.option_chain(self.exp_date)
        else:            
            print(' Call option Expiration Date from option data: ',stk.options[0])
            opt = stk.option_chain(stk.options[0]) # If input is not provided, take the latest option expiration date to extract call options
            
        
        option_calls = opt.calls
        
        # Only call option OTM(Out of the money) are selected with non zero bid price
        options = option_calls.loc[option_calls['inTheMoney'] == False] 
        options = options.loc[options['bid'] > 0]   
                
        # Matirity price ranges from current price-50$ to maximum strike price listed in OTM call options. simulations are considered.
        # simulations value can be changed to any number of simulations
        simulations = 20
        bound = 100   # maturity prices for 100$ less than current price and 100$ more than maximum strike price
        st = np.linspace(s0-bound, int(max(options['strike']))+bound, simulations, endpoint=True) # Maturity prices in the acsending order
        
        # minimum strike price and maximum strike price from call options
        strike_min = min(options['strike'])
        strike_max = max(options['strike'])
        
        # picking the first 'n' strike prices and bid prices from call options for which option price is higher                
        k = options['strike'][:n] 
        k = np.array(k)
        C = options['bid'][:n]   
        C = np.array(C)
        
        # If there are less than n number of call options, add 0.0 values to the rest of the strike and bid prices
        
        if len(k) or len(C) <= n:
            add_values = n - len(k)            
            k = np.concatenate([np.zeros(add_values), k])
            C = np.concatenate([np.zeros(add_values), C])
            
        
        ################Print parameters#####################
        print('stock position', self.N)
        print('Number of groups with varying contract sizes', n)
        print('Current market price', s0)
        print('Maturity price at expiration date', st)
        print('Best bid price', C)
        print('Strike Price', k)
        print('min range for strike price',strike_min)
        print('max range for strike price',strike_max)
        ################################
        
                
        # Random grouping of contract sizes for the given stock position
        #contract_sizes = constrained_sum_sample_pos(n, self.N) 
        p = [random.random() for i in range(0,5)]
        s = sum(p)
        p = [ i/s for i in p ]
        contract_sizes = p
        print('Initial probabilities', contract_sizes)       
        # Setting initial value with random grouping of contract sizes   
        x0 = contract_sizes                      
        
        for i in range(0, len(st)):  # Loop with all simulated maturity prices for the latest option expiration date
            
            # Constraint to match the total contract sizes to given stock position     
            #cons1 = ({'type': 'eq', 'fun': lambda x: np.sum(x) - self.N})
            cons1 = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        
            # Contract sizes in each group within the bounds of minimum 1 unit to maximum number of stock position. 
            # Minimum bound as 1 unit has limitation that if a stock has less than 'n' best strike prices, 0 strike price is alloted for 
            # rest of n cases
            # If minimum bound is 0 unit, algorithm allocates all contracts to one strike price depends on maturity price
            #bnds1 = ((1,self.N),)*n
            bnds1 = ((0,1),)*n
            
            '''
            # Input arguments for minimize nagative expected returns from call options are ---
            # Stock prices at maturity dates(st)
            # current stock price (s0)
            # Strike price (k)
            # bid price (C)
            '''
            # Maximum difference between Maturity price and strike price is taken. That means, if strike price is more than 
            # maturity price, difference is 0
            
            diff = []
            diff[:] = [st[i] - l for l in k]
                        
            for num in range(0, len(diff)):
                if diff[num] < 0:
                    diff[num] = 0            
                      
            args = (st[i], s0, diff, C, self.N)
        
            # Objective function is formulated as minimizing the negative of maximum expected return
                    
            res = minimize(Objective_func, x0, args=args,method='SLSQP',
                  bounds=bnds1, constraints=cons1) 
                           #options={'disp': True, 'eps' : 1e-8, 'maxiter':1000})
            
            print('++++++++++')
            print("Result",res)
            print('Best grouping of contract sizes', res.x)
            print('Maturity price', st[i])     
            print('strike price', k)
            print('bid price', C)
            expected_returns.append(-(res.fun/100)*s0*self.N)
            #contractsize.append([int(round(res.x,0)) for res.x in res.x])
            contractsize.append(res.x)
            strike.append(k)
            bid.append(C)
            maturity.append(st[i])
            currentprice.append(s0)
               
                    
        return  
       

In [66]:
   

if __name__ == '__main__':
    global contractsize, strike, bid, maturity, expected_returns, currentprice
    contractsize = []
    strike = []
    expected_returns = []
    maturity = []
    bid = []
    currentprice = []
    '''
    
    # print documentation
     
    print(__doc__) # print all comments in the scipt
    
    print ('Number of arguments:', len(sys.argv), 'arguments.')
    print ('Argument List:', str(sys.argv))
    
    
    def mkdate(datestr):
        return time.strptime(datestr, '%Y-%m-%d')

      # Code to use command line arguments
    parser = argparse.ArgumentParser()
    parser.add_argument('-s', "--stksymbol", help="stock symbol such as 'aapl' ", required=True)
    parser.add_argument('-p', "--stkposition", type=int, help="Current stock position- one contract size is 100 shares. Should be minimum 500", required=True)
    parser.add_argument('-e', "--expdate", type=mkdate, help='The Start Date - format YYYY-MM-DD', required=True)
    args = parser.parse_args(sys.argv[1:])
    print('args')
    print(args.stksymbol)
    print(args.stkposition)
    print(args.expdate)
    print(args)
    print(args.expdate.tm_mon)
    
    '''
    args = {'stksymbol': 'mmm',
             'stkposition': 1000,  # Input stock position should be > 500 due to the limitation that n is 5
             'expdate': '2021-06-22'
             }   
    today = date.today()
    print("Today's date:", today)
    
    q, mod = divmod(args['stkposition'], 100)
        
    if mod !=0:
        print('one contract size is 100 shares. No. of contracts can be created are only.. ',q)
    
    
    args['stkposition'] = q
    if q < 5:
        print('Input stock position should be greater than or equal to 500')
    else:
        best_options = MaximizeMarginUtilization(args)
        best_options.run()
    
    

Today's date: 2021-06-01
{'stksymbol': 'mmm', 'stkposition': 10, 'expdate': '2021-06-22'}
 Call option Expiration Date from option data:  2021-06-04
stock position 10
Number of groups with varying contract sizes 5
Current market price 203.0399932861328
Maturity price at expiration date [103.03999329 114.56420417 126.08841505 137.61262593 149.1368368
 160.66104768 172.18525856 183.70946944 195.23368032 206.7578912
 218.28210208 229.80631296 241.33052384 252.85473472 264.3789456
 275.90315648 287.42736736 298.95157824 310.47578912 322.        ]
Best bid price [0.55 0.12 0.07 0.05 0.04]
Strike Price [205.  207.5 210.  212.5 215. ]
min range for strike price 205.0
max range for strike price 222.5
Initial probabilities [0.1212621399225017, 0.3174728544152059, 0.004944663484976675, 0.32520152590508156, 0.23111881627223402]
10 [0.12126214 0.31747285 0.00494466 0.32520153 0.23111882] [0, 0, 0, 0, 0] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 103.03999328613281 -998.6935812512271
10 [0.121262

10 [0.12126214 0.31747285 0.00494466 0.32520154 0.23111882] [0, 0, 0, 0, 0] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 160.6610476845189 -422.48303725991553
10 [0.12126214 0.31747285 0.00494466 0.32520153 0.23111883] [0, 0, 0, 0, 0] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 160.6610476845189 -422.4830372614056
10 [2.99803229e-01 2.84232919e-01 1.73472348e-17 2.57485787e-01
 1.58478065e-01] [0, 0, 0, 0, 0] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 160.6610476845189 -421.60732463275485
10 [2.99803229e-01 2.84232919e-01 1.73472348e-17 2.57485787e-01
 1.58478065e-01] [0, 0, 0, 0, 0] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 160.6610476845189 -421.60732463275485
10 [2.99803244e-01 2.84232919e-01 1.73472348e-17 2.57485787e-01
 1.58478065e-01] [0, 0, 0, 0, 0] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 160.6610476845189 -421.60732455079847
10 [2.99803229e-01 2.84232934e-01 1.73472348e-17 2.57485787e-01
 1.58478065e-01] [0, 0, 0, 0, 0] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 1

 2.24301710e-17] [1.7578912032277856, 0, 0, 0, 0] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 206.75789120322779 38.30827972915818
10 [0.00000000e+00 8.97035213e-01 6.87949339e-03 9.60853085e-02
 2.24301710e-17] [1.7578912032277856, 0, 0, 0, 0] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 206.75789120322779 38.30827972617794
10 [0.00000000e+00 8.97035213e-01 6.87949339e-03 9.60852936e-02
 1.49011612e-08] [1.7578912032277856, 0, 0, 0, 0] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 206.75789120322779 38.308279724687836
10 [0.00000000e+00 1.00000000e+00 0.00000000e+00 4.16333634e-17
 5.76513088e-16] [1.7578912032277856, 0, 0, 0, 0] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 206.75789120322779 38.37897917094973
10 [0.00000000e+00 1.00000000e+00 0.00000000e+00 4.16333634e-17
 5.76513088e-16] [1.7578912032277856, 0, 0, 0, 0] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 206.75789120322779 38.37897917094973
10 [1.49011612e-08 1.00000000e+00 0.00000000e+00 4.16333634e-17
 5.76513088e-16] [1

10 [0.12126215 0.31747285 0.00494466 0.32520153 0.23111882] [105.47578912032276, 102.97578912032276, 100.47578912032276, 97.97578912032276, 95.47578912032276] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 310.47578912032276 76.59252085701752
10 [0.12126214 0.31747287 0.00494466 0.32520153 0.23111882] [105.47578912032276, 102.97578912032276, 100.47578912032276, 97.97578912032276, 95.47578912032276] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 310.47578912032276 76.59252116547151
10 [0.12126214 0.31747285 0.00494468 0.32520153 0.23111882] [105.47578912032276, 102.97578912032276, 100.47578912032276, 97.97578912032276, 95.47578912032276] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 310.47578912032276 76.59252153055007
10 [0.12126214 0.31747285 0.00494466 0.32520154 0.23111882] [105.47578912032276, 102.97578912032276, 100.47578912032276, 97.97578912032276, 95.47578912032276] [0.55 0.12 0.07 0.05 0.04] 203.0399932861328 310.47578912032276 76.5925219000988
10 [0.12126214 0.31747285 0.00494466 

# Observations from Testing


Testing -


1. At different stock maturity prices ranging from current market price to maximum strike price listed in call options, It has been observed that algorithm is allocating more contracts to highest strike prices and lowest bid price combination that yield minimum negative returns.

That means in the above example,  highest contract size (6 units) are assigned to highest strike price (90) and lowest bid price (33.8).

2. As Maturity prices increases, objective function returns are increasing.



In [37]:
contractsize

[[1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1]]

In [67]:
# with Strike price average of option strike prices
output = pd.DataFrame(list(zip(contractsize, strike, bid, maturity, currentprice, expected_returns)),
               columns =['contract_sizes', 'strike_prices', 'bid_prices', 'maturity_price', 'current_price', 'expected_returns'])


In [50]:
output.to_excel(r'CoveredCallResults\coveredcallprob\Optimization_prob_appl2.xlsx', index = False)

In [68]:
output

,contract_sizes,strike_prices,bid_prices,maturity_price,current_price,expected_returns
0,"[1.0, 0.0, 0.0, 0.0, 0.0]","[205.0, 207.5, 210.0, 212.5, 215.0]","[0.55, 0.12, 0.07, 0.05, 0.04]",103.039993,203.039993,-994.500000
1,"[0.9999999999999966, 9.853229343548264e-16, 2....","[205.0, 207.5, 210.0, 212.5, 215.0]","[0.55, 0.12, 0.07, 0.05, 0.04]",114.564204,203.039993,-879.257891
2,"[1.0, 0.0, 0.0, 0.0, 0.0]","[205.0, 207.5, 210.0, 212.5, 215.0]","[0.55, 0.12, 0.07, 0.05, 0.04]",126.088415,203.039993,-764.015782
3,"[0.9999999999999983, 4.0245584642661925e-16, 0...","[205.0, 207.5, 210.0, 212.5, 215.0]","[0.55, 0.12, 0.07, 0.05, 0.04]",137.612626,203.039993,-648.773674
4,"[0.9999999999999988, 0.0, 3.0531133177191805e-...","[205.0, 207.5, 210.0, 212.5, 215.0]","[0.55, 0.12, 0.07, 0.05, 0.04]",149.136837,203.039993,-533.531565
5,"[1.0, 0.0, 1.734723475976807e-16, 0.0, 0.0]","[205.0, 207.5, 210.0, 212.5, 215.0]","[0.55, 0.12, 0.07, 0.05, 0.04]",160.661048,203.039993,-418.289456
6,"[1.0, 0.0, 4.85722573273506e-17, 0.0, 0.0]","[205.0, 207.5, 210.0, 212.5, 215.0]","[0.55, 0.12, 0.07, 0.05, 0.04]",172.185259,203.039993,-303.047347
7,"[0.9999999999999997, 0.0, 1.1102230246251565e-...","[205.0, 207.5, 210.0, 212.5, 215.0]","[0.55, 0.12, 0.07, 0.05, 0.04]",183.709469,203.039993,-187.805238
8,"[1.0, 0.0, 1.8735013540549517e-16, 0.0, 0.0]","[205.0, 207.5, 210.0, 212.5, 215.0]","[0.55, 0.12, 0.07, 0.05, 0.04]",195.233680,203.039993,-72.563130
9,"[0.0, 1.0, 0.0, 4.163336342344337e-17, 5.76513...","[205.0, 207.5, 210.0, 212.5, 215.0]","[0.55, 0.12, 0.07, 0.05, 0.04]",206.757891,203.039993,38.378979


In [44]:
stk = yf.Ticker('aapl') # Passing stock stymbo
op = stk.option_chain('2021-06-04')
op.calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL210604C00070000,2021-05-24 19:55:17,70.0,57.25,52.95,57.00,0.000000,0.000000,1.0,24,2.980471,True,REGULAR,USD
1,AAPL210604C00075000,2021-05-24 14:41:38,75.0,52.50,48.10,52.00,0.000000,0.000000,8.0,40,2.759769,True,REGULAR,USD
2,AAPL210604C00080000,2021-05-26 17:05:08,80.0,46.85,43.80,45.50,0.000000,0.000000,3.0,22,1.718751,True,REGULAR,USD
3,AAPL210604C00085000,2021-05-11 15:59:48,85.0,41.61,38.80,40.50,0.000000,0.000000,1.0,13,1.500002,True,REGULAR,USD
4,AAPL210604C00090000,2021-05-28 19:58:48,90.0,34.70,33.80,35.50,-2.450001,-6.594887,73.0,619,1.296879,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,AAPL210604C00180000,2021-05-07 18:53:35,180.0,0.01,0.00,0.01,0.000000,0.000000,15.0,86,1.125004,False,REGULAR,USD
63,AAPL210604C00185000,2021-05-04 13:35:17,185.0,0.01,0.00,0.01,0.000000,0.000000,2.0,18,1.218754,False,REGULAR,USD
64,AAPL210604C00190000,2021-05-24 13:30:05,190.0,0.01,0.00,0.01,0.000000,0.000000,1.0,13,1.281254,False,REGULAR,USD
65,AAPL210604C00195000,2021-05-03 15:46:47,195.0,0.02,0.00,0.01,0.000000,0.000000,1.0,1,1.375003,False,REGULAR,USD


In [47]:
OTM = op.calls.loc[op.calls['inTheMoney'] == False]

In [48]:
OTM

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
26,AAPL210604C00125000,2021-05-28 19:59:59,125.0,1.02,0.97,1.08,-0.63,-38.181816,24424.0,4565,0.242683,False,REGULAR,USD
27,AAPL210604C00126000,2021-05-28 19:59:59,126.0,0.63,0.60,0.64,-0.52,-45.217392,29883.0,9368,0.231209,False,REGULAR,USD
28,AAPL210604C00127000,2021-05-28 19:59:58,127.0,0.36,0.34,0.38,-0.39,-52.000000,22046.0,20213,0.232430,False,REGULAR,USD
29,AAPL210604C00128000,2021-05-28 19:59:59,128.0,0.21,0.20,0.22,-0.27,-56.250000,21551.0,9743,0.236336,False,REGULAR,USD
30,AAPL210604C00129000,2021-05-28 19:59:51,129.0,0.13,0.11,0.14,-0.18,-58.064520,8527.0,8439,0.248054,False,REGULAR,USD
31,AAPL210604C00130000,2021-05-28 20:00:00,130.0,0.09,0.08,0.09,-0.11,-55.000000,16048.0,15340,0.260750,False,REGULAR,USD
32,AAPL210604C00131000,2021-05-28 19:59:57,131.0,0.06,0.05,0.06,-0.08,-57.142853,9543.0,6602,0.273445,False,REGULAR,USD
33,AAPL210604C00132000,2021-05-28 19:59:59,132.0,0.05,0.04,0.05,-0.04,-44.444447,2462.0,9344,0.296882,False,REGULAR,USD
34,AAPL210604C00133000,2021-05-28 19:58:46,133.0,0.04,0.03,0.04,-0.04,-50.000000,3350.0,5655,0.316413,False,REGULAR,USD
35,AAPL210604C00134000,2021-05-28 19:59:28,134.0,0.03,0.03,0.04,-0.03,-50.000000,1513.0,4919,0.347663,False,REGULAR,USD
